In [ ]:
import numpy as np
import librosa
import matplotlib.pyplot as plt
import pretty_midi
import midi
import matplotlib.patches
import matplotlib
import djitw
import scipy.spatial
import librosa
import sys
import os
import csv
import glob
import tabulate
import ujson as json
import collections
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')
sns.set_style({'grid.linewidth': 1.3})
matplotlib.rc('font',**{'size':13, 'family':'Open Sans'})

In [ ]:
# Testing colors
BLUE = '#1a6396'
GREEN = '#59dd97'
ORANGE = '#E8B71A'
GREY = '#DFDFDF'
RED = '#DB3340'
TAN = '#F7EAC8'
FIGSIZE = (9, 6)
FIGSIZE_FLAT = (9, 2)
plt.figure(figsize=(8, 3))
plt.gca().add_patch(plt.Rectangle((-1, -1), 8, 1.5, fc=TAN, lw=0))
plt.gca().add_patch(plt.Rectangle((0, 0), 1, 1, fc=BLUE, lw=0))
plt.gca().add_patch(plt.Rectangle((1, 0), 1, 1, fc=GREEN, lw=0))
plt.gca().add_patch(plt.Rectangle((2, 0), 1, 1, fc=GREY, lw=0))
plt.gca().add_patch(plt.Rectangle((3, 0), 1, 1, fc=RED, lw=0))
plt.gca().add_patch(plt.Rectangle((4, 0), 1, 1, fc=ORANGE, lw=0))
plt.gca().add_patch(plt.Rectangle((5, 0), 1, 1, fc=TAN, lw=0))
plt.gca().add_patch(plt.Rectangle((5, 0), 1, 1, fc=TAN, lw=0))
plt.xlim([-1, 7])
plt.ylim([-1, 2])
#plt.axis('off')

# Chapter 1

In [ ]:
plt.figure(figsize=(FIGSIZE[0], FIGSIZE[1]*2))
ax = plt.gca()
t = np.linspace(.1, .9, 9)
plt.vlines(t, 0, 1., linestyles='dashed', alpha=.3, zorder=-1)

vc = .9
words = 'The quick brown fox jumps over the lazy dog'.split(' ')
for x, word in zip(t, words):
    ax.text(x, vc, word, {'family': 'monospace', 'size': 12}, ha='center', va='center')

vc = .7
signal = .08*np.sin(2.4*t*np.pi) + vc
plt.plot(t, signal, 'k.', ms=10)
plt.vlines(t, [s if s > vc else vc for s in signal],
           [s if s < vc else vc for s in signal], lw=1.2) 

vc = .5
a, _ = librosa.load('data/1_A.wav')
N = a.shape[0]
for x in t:
    frame = a[(x - .1)*N:(x + .1)*N]
    spectrum = np.abs(np.fft.rfft(frame))
    spectrum = spectrum[:spectrum.shape[0]/3]
    spectrum = spectrum/3000.
    plt.plot(x + spectrum, np.linspace(vc - .08, vc + .08, spectrum.shape[0]), 'k')

axis = plt.axis()
vc = .3
w = .02
h = .08
dna_names = ['T', 'A', 'C', 'G']
for x in t:
    dna = np.zeros((4, 1))
    n = np.random.randint(0, 4)
    dna[n] = 1
    plt.imshow(dna, interpolation='nearest', extent=(x - w, x + w, vc - h/2, vc + h), cmap=plt.cm.gray)
    plt.plot((x - w, x - w, x + w, x + w, x - w), (vc - h/2, vc + h, vc + h, vc - h/2, vc - h/2), 'k')
    ax.text(x, vc - h/2 - .01, dna_names[n], {'family': 'monospace', 'size': 16}, ha='center', va='top')
plt.axis(axis)

vc = .1
for n, x in enumerate(t):
    im = plt.imread('data/1_video/{}.png'.format(n + 1))
    plt.imshow(im, interpolation='nearest', extent=(x - .03, x + .03, vc - .08, vc + .08))

plt.xlim([0.05, 0.95])
plt.ylim([0, 1])
plt.yticks([])
plt.axis('off')

vc = 0.
words = 'The quick brown fox jumps over the lazy dog'.split(' ')
for n, x in enumerate(t):
    ax.text(x, vc, '$t_{}$'.format(n), {'family': 'monospace', 'size': 16}, ha='center', va='top')

plt.savefig('1-example_sequences.pdf', bbox_inches='tight', pad_inches=0.)

In [ ]:
np.random.seed(7)
match_length = 100
crop = match_length/5
def random_walk(N):
    return np.cumsum(np.random.random_integers(-1, 1, N))/np.log(N)
def random_sine(N):
    return np.sin(np.linspace(0, 5*np.pi, N)*np.random.uniform(.9, 1.1) + np.random.uniform(0, 2*np.pi))

match = random_sine(match_length + match_length/10)
query = np.interp(np.arange(match_length),
                  np.arange(match_length + match_length/10),
                  match + .5*random_walk(match_length + match_length/10))
match = match[match_length/10:] + .5*random_walk(match_length)
match = (match - match.mean())/match.std()
query = (query - query.mean())/query.std()

D = np.subtract.outer(match, query[crop/2:-crop/2])**2
p, q, score = djitw.dtw(D, inplace=False)

In [ ]:
ds = 3

plt.figure(figsize=FIGSIZE)
plt.plot(match - match.min(), GREEN, lw=2)
plt.plot(query - query.max(), BLUE, lw=2)

for n in range(0, match_length, ds) + [match_length - 1]:
    plt.plot([n, n], [match[n] - match.min(), query[n] - query.max()], 'k:', lw=2)
    
plt.xlim(-1, plt.axis()[1])
plt.axis('off')
plt.savefig('1-example_distance_unwarped.pdf', bbox_inches='tight', pad_inches=0.1)

plt.figure(figsize=FIGSIZE)
plt.plot(match - match.min(), GREEN, lw=2)
plt.plot(np.arange(crop/2, match_length - crop/2),
         query[crop/2:-crop/2] - query[crop/2:-crop/2].max(), BLUE, lw=2)

for p_n, q_n in zip(p[::ds], q[::ds]):
    
    plt.plot([p_n, q_n + crop/2],
             [match[p_n] - match.min(), query[crop/2:-crop/2][q_n] - query[crop/2:-crop/2].max()],
             'k:', lw=2)

plt.plot([p[-1], q[-1] + crop/2],
         [match[p[-1]] - match.min(), query[crop/2:-crop/2][q[-1]] - query[crop/2:-crop/2].max()],
         'k:', lw=2)

plt.xlim(-1, plt.axis()[1])
plt.axis('off')
plt.savefig('1-example_distance_warped.pdf', bbox_inches='tight', pad_inches=0.1)

# Chapter 2

In [ ]:
def draw_neural_net(ax, left, right, bottom, top, layer_sizes):
    '''
    Draw a neural network cartoon using matplotilb.
    
    :usage:
        >>> fig = plt.figure(figsize=(12, 12))
        >>> draw_neural_net(fig.gca(), .1, .9, .1, .9, [4, 7, 2])
    
    :parameters:
        - ax : matplotlib.axes.AxesSubplot
            The axes on which to plot the cartoon (get e.g. by plt.gca())
        - left : float
            The center of the leftmost node(s) will be placed here
        - right : float
            The center of the rightmost node(s) will be placed here
        - bottom : float
            The center of the bottommost node(s) will be placed here
        - top : float
            The center of the topmost node(s) will be placed here
        - layer_sizes : list of int
            List of layer sizes, including input and output dimensionality
    '''
    n_layers = len(layer_sizes)
    v_spacing = (top - bottom)/float(max(layer_sizes))
    h_spacing = (right - left)/float(len(layer_sizes) - 1)
    # Nodes
    for n, layer_size in enumerate(layer_sizes):
        layer_top = v_spacing*(layer_size - 1)/2. + (top + bottom)/2.
        for m in xrange(layer_size):
            circle = plt.Circle((n*h_spacing + left, layer_top - m*v_spacing), v_spacing/4.,
                                color='w', ec='k', zorder=4)
            ax.add_artist(circle)
    # Edges
    for n, (layer_size_a, layer_size_b) in enumerate(zip(layer_sizes[:-1], layer_sizes[1:])):
        layer_top_a = v_spacing*(layer_size_a - 1)/2. + (top + bottom)/2.
        layer_top_b = v_spacing*(layer_size_b - 1)/2. + (top + bottom)/2.
        for m in xrange(layer_size_a):
            for o in xrange(layer_size_b):
                line = plt.Line2D([n*h_spacing + left, (n + 1)*h_spacing + left],
                                  [layer_top_a - m*v_spacing, layer_top_b - o*v_spacing], c='k')
                ax.add_artist(line)

def label(ax, x, y, text, label_size=30, **kwargs):          
    plt.text(x, y, text, {'size': label_size},
             va='center', ha='center', zorder=10, **kwargs)

def heaviside(ax, left, right, bottom, top):
    middle = left + (right - left)/2.
    for x, y in zip([[left, middle], [middle, middle], [middle, right]], 
                    [[bottom, bottom], [bottom, top], [top, top]]):
        line = plt.Line2D(x, y, c='k', lw=2, zorder=10)
        ax.add_artist(line)

plt.figure(figsize=(FIGSIZE[0], (1 - .16)*FIGSIZE[0]))
ax = plt.gca()
draw_neural_net(ax, .1, .9, .0, 1., [3, 1])

label(ax, .1, .83333, '$x[1]$')
label(ax, .1, .5, '$x[2]$')
label(ax, .1, .16666, '$x[3]$')

label(ax, .5, .71, '$w[1]$', rotation=-24)
label(ax, .5, .535, '$w[2]$')
label(ax, .5, .375, '$w[3]$', rotation=24)

b_coords = (.6, .2)
circle = plt.Circle(b_coords, 1./3./4.,
                    color='w', ec='k', zorder=4)
plt.gca().add_artist(circle)
label(ax, b_coords[0], b_coords[1], '$b$', 36)
line = plt.Line2D([b_coords[0], .9], [b_coords[1], .5], c='k')
ax.add_artist(line)

heaviside(ax, .842, .958, .46, .54)

plt.ylim([.08, .92])
plt.axis('off')
plt.savefig('2-perceptron.pdf', bbox_inches='tight', pad_inches=0.1)

In [ ]:
plt.figure(figsize=(FIGSIZE[0], .71*FIGSIZE[0]))
ax = plt.gca()

circle_radius = .04
circle_spacing = .09

def circle_grid(ax, size, left, bottom, color='w', zorder=4, **kwargs):
    for x in range(size[0]):
        for y in range(size[1]):
            circle = plt.Circle((x*circle_spacing + left, y*circle_spacing + bottom),
                                circle_radius, color=color, ec='k', zorder=zorder - x - y, **kwargs)
            ax.add_artist(circle)

input_left, input_bottom = .05, .05
output_left, output_bottom = .65, input_bottom + 3.85*circle_spacing
circle_grid(ax, (6, 6), input_left, input_bottom)
circle_grid(ax, (4, 4), output_left, output_bottom, color='none', zorder=35, alpha=.3)
circle_grid(ax, (3, 3), input_left + 2*circle_spacing, input_bottom + 2*circle_spacing, GREEN, zorder=30)
circle_grid(ax, (1, 1), output_left + 2*circle_spacing, output_bottom + 2*circle_spacing, GREEN, zorder=40)

ffdeg = circle_radius*np.sqrt(2)/2
for x in range(3):
    for y in range(3):
        line = plt.Line2D([input_left + 2*circle_spacing + x*circle_spacing,
                           output_left + 2*circle_spacing],
                          [input_bottom + 2*circle_spacing + y*circle_spacing,
                           output_bottom + 2*circle_spacing], c='k', zorder=30 - x - y - 1)
        ax.add_artist(line)
plt.axis([0, .97, 0, .71])
plt.axis('off')
plt.savefig('2-convolution.pdf', bbox_inches='tight', pad_inches=0.1)

In [ ]:
plt.figure(figsize=FIGSIZE)
def f(x):
    return x**2
def df(x):
    return 2*x
x = np.linspace(-1, 1, 100)
x_init = .7

for n, lr in enumerate([1.035, .9, .4, .08]):
    plt.subplot(2, 2, n + 1)
    plt.plot(x, f(x), BLUE, lw=2)
    x_current = x_init
    for i in range(5):
        x_new = x_current - lr*df(x_current)
        plt.plot([x_current, x_new], [f(x_current), f(x_new)], 'k:')
        plt.plot([x_current, x_new], [f(x_current), f(x_new)], 'k.', ms=10)
        x_current = x_new
    plt.ylim(-.01, 1.)
    plt.axis('off')
plt.savefig('2-learning_rate.pdf', bbox_inches='tight', pad_inches=0.1)

In [ ]:
import bayes_opt
import matplotlib.gridspec

def target(x):
    return (-3*(x - .7)**2) + .3*(np.sin(10*x**2) + 3)

x = np.linspace(0, 1, 1000)
y = target(x)

bo = bayes_opt.BayesianOptimization(target, {'x': (0, 1)}, False)

gp_params = {'corr': 'squared_exponential'}
bo.initialize(dict((target(n), {'x': n}) for n in [.05, .4, .7, .97]))
bo.maximize(init_points=0, n_iter=0, acq='ei', **gp_params)

def posterior(bo, xmin=-2, xmax=10):
    xmin, xmax = 0, 1
    bo.gp.fit(bo.X, bo.Y)
    mu, sigma2 = bo.gp.predict(np.linspace(xmin, xmax, 1000).reshape(-1, 1), eval_MSE=True)
    return mu, np.sqrt(sigma2)

fig = plt.figure(figsize=FIGSIZE)

gs = matplotlib.gridspec.GridSpec(2, 1, height_ratios=[3, 1]) 
ax = plt.subplot(gs[0])

mu, sigma = posterior(bo)
ax.plot(x, y, BLUE, linewidth=2)
ax.plot(bo.X.flatten(), bo.Y, 'k.', markersize=20)

ax.fill(np.concatenate([x, x[::-1]]), 
          np.concatenate([mu - sigma, (mu + sigma)[::-1]]),
          fc=GREY, ec='None')
ax.axis('off')
ax.set_title('Objective')
ax.set_ylim([y.min(), y.max()*1.1])

ax.set_xlim((0, 1))

ax = plt.subplot(gs[1])
utility = bo.util.utility(x.reshape((-1, 1)), bo.gp, 0)
ax.fill_between(x, utility, facecolor=GREEN, edgecolor='none')
ax.set_xlim((0, 1))
ax.axis('off')
ax.set_title('Expected Improvement')
plt.savefig('2-bayesian_optimization.pdf', bbox_inches='tight', pad_inches=0.1)

In [ ]:
duration = 5
a, fs = librosa.load('data/2_song.mp3', offset=.25, duration=duration)
plt.figure(figsize=FIGSIZE)
plot_fs = 1000
plt.plot(np.linspace(0, duration, duration*plot_fs), librosa.resample(a, fs, plot_fs), BLUE, lw=.2)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.ylim([-1, 1])
plt.savefig('2-audio_signal.pdf', bbox_inches='tight', pad_inches=0.1)

In [ ]:
import scipy.interpolate

tbl_f = np.asarray(
    [20, 25, 31.5, 40, 50, 63, 80, 100, 125, 160, 200, 250, 315, 400,
     500, 630, 800, 1000, 1250, 1600, 2000, 2500, 3150, 4000, 5000, 6300,
     8000, 10000, 12500])
tbl_alpha_f = np.asarray(
    [0.532, 0.506, 0.480, 0.455, 0.432, 0.409, 0.387, 0.367, 0.349, 0.330,
     0.315, 0.301, 0.288, 0.276, 0.267, 0.259, 0.253, 0.250, 0.246, 0.244,
     0.243, 0.243, 0.243, 0.242, 0.242, 0.245, 0.254, 0.271, 0.301])
tbl_L_U = np.asarray(
    [-31.6, -27.2, -23.0, -19.1, -15.9, -13.0, -10.3, -8.1, -6.2, -4.5,
     -3.1, -2.0, -1.1, -0.4, 0.0, 0.3, 0.5, 0.0, -2.7, -4.1, -1.0, 1.7,
     2.5, 1.2, -2.1, -7.1, -11.2, -10.7, -3.1])
tbl_T_f = np.asarray(
    [78.5, 68.7, 59.5, 51.1, 44.0, 37.5, 31.5, 26.5, 22.1, 17.9, 14.4,
     11.4, 8.6, 6.2, 4.4, 3.0, 2.2, 2.4, 3.5, 1.7, -1.3, -4.2, -6.0, -5.4,
     -1.5, 6.0, 12.6, 13.9, 12.3])

def iso226_spl_contour(f, L_N=40):
    A_f = (4.47E-3*(10.0**(0.025*L_N)-1.15) +
           (0.4*10.0**((tbl_T_f+tbl_L_U)/10.0-9.0))**tbl_alpha_f)
    return scipy.interpolate.InterpolatedUnivariateSpline(
        tbl_f, (10.0/tbl_alpha_f)*np.log10(A_f) - tbl_L_U + 94.0, k=3)(f)

f = np.logspace(np.log10(20), np.log10(12500), 100, base=10)
l = iso226_spl_contour(f)
plt.figure(figsize=FIGSIZE)
plt.semilogx(f, l, BLUE, lw=2)
plt.xlim([15, 15000])
plt.xticks([30, 100, 300, 1000, 3000, 10000],
           [30, 100, 300, 1000, 3000, 10000])
plt.xlabel('Frequency (Hz)')
plt.ylabel('Sound Pressure Level (dB)')
plt.savefig('2-equal_loudness.pdf', bbox_inches='tight', pad_inches=0.1)

In [ ]:
piano = pretty_midi.PrettyMIDI()
i = pretty_midi.Instrument(0, False)
i.notes.append(pretty_midi.Note(100, 48, 0., 1.))
piano.instruments.append(i)

plt.figure(figsize=FIGSIZE)
plt.plot(piano.fluidsynth(8000)[400:600], BLUE, lw=2)

guitar = pretty_midi.PrettyMIDI()
i = pretty_midi.Instrument(41, False)
i.notes.append(pretty_midi.Note(100, 48, 0., 1.))
piano.instruments.append(i)
plt.plot(piano.fluidsynth(8000)[400:600], GREEN, lw=2)
sns.despine()
plt.xlabel('Time (ms)')
plt.ylabel('Amplitude')
plt.ylim([-1, 1])
plt.xticks(np.arange(0, 201, 50), np.arange(0, 201, 50)/8.)
plt.savefig('2-timbres.pdf', bbox_inches='tight', pad_inches=0.1)

In [ ]:
plt.figure(figsize=FIGSIZE)
A = np.abs(np.fft.rfft(a))
N = A.shape[0]
#A /= N
A = A[:5001]
plt.xticks(np.arange(0, A.shape[0], A.shape[0]/4),
           100*np.ceil(22050*np.arange(0, A.shape[0], A.shape[0]/4)/N/100.).astype(int))
plt.plot(A, BLUE, lw=.5)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')
plt.savefig('2-spectrum.pdf', bbox_inches='tight', pad_inches=0.1)

with sns.axes_style('white'):
    plt.figure(figsize=FIGSIZE_FLAT)
    A = np.abs(librosa.stft(a))[:93]
    plt.imshow(A, aspect='auto', origin='lower', cmap=plt.cm.hot, interpolation='none',
               vmin=np.percentile(A, 10), vmax=np.percentile(A, 99.5))
    plt.yticks(np.arange(0, A.shape[0], A.shape[0]/4),
               100*np.ceil(22050*np.arange(0, A.shape[0], A.shape[0]/4)/1024./100.).astype(int))
    plt.xticks(np.arange(0, A.shape[1], A.shape[1]/5), np.arange(6))
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (Hz)')
    plt.savefig('2-spectrogram.pdf', bbox_inches='tight', pad_inches=0.1)

    plt.figure(figsize=FIGSIZE_FLAT)
    A = librosa.logamplitude(A)
    plt.imshow(A, aspect='auto', origin='lower', cmap=plt.cm.hot, interpolation='none',
               vmin=np.percentile(A, 10), vmax=np.percentile(A, 99.5))
    plt.yticks(np.arange(0, A.shape[0], A.shape[0]/4),
               100*np.ceil(22050*np.arange(0, A.shape[0], A.shape[0]/4)/1024./100.).astype(int))
    plt.xticks(np.arange(0, A.shape[1], A.shape[1]/5), np.arange(6))
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (Hz)')
    plt.savefig('2-log_spectrogram.pdf', bbox_inches='tight', pad_inches=0.1)

    plt.figure(figsize=FIGSIZE_FLAT)
    A = librosa.logamplitude(np.abs(librosa.cqt(a, fmin=librosa.midi_to_hz(36), n_bins=48, real=False)))
    plt.imshow(A, aspect='auto', origin='lower', cmap=plt.cm.hot, interpolation='none',
               vmin=np.percentile(A, 10), vmax=np.percentile(A, 99.9))
    plt.yticks(range(0, 48, 12), [librosa.midi_to_note(n) for n in range(36, 36 + 48, 12)])
    plt.xticks(np.arange(0, A.shape[1], A.shape[1]/5), np.arange(6))
    plt.xlabel('Time (s)')
    plt.ylabel('Note')
    plt.savefig('2-cqt.pdf', bbox_inches='tight', pad_inches=0.1)

In [ ]:
signal1 = np.array([509, 113, -229, 253, -96, -195, 180, -303, -361, 17,
                    -13, 242, 14, -230, 300, 89, -112, -236, -298])
signal2 = np.array([543, 401, 122, -288, 62, 259, 180, -72, -336, 10,
                    223, 263, 35, -345, 68, 400, 38, -109, -301])
q = [0, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 8, 9, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 18]
p = [0, 0, 1, 2, 3, 3, 3, 4, 5, 6, 7, 8, 9, 10, 11, 11, 12, 13, 14, 14, 15, 16, 17, 18]
with sns.axes_style('ticks'):
    plt.figure(figsize=FIGSIZE)
    plt.plot(signal1, lw=2, c=BLUE)
    plt.plot(signal1, '.', ms=10, c=BLUE)
    plt.plot(signal2 + 1000, lw=2, c=GREEN)
    plt.plot(signal2 + 1000, '.', ms=10, c=GREEN)
    for p_n, q_n in zip(p, q):
        plt.plot([p_n, q_n],  [signal1[p_n], signal2[q_n] + 1000], 'k:', lw=2)
    ax = plt.gca()
    ax.get_yaxis().set_visible(False)
    sns.despine(left=True)
    plt.xticks(range(0, 19, 3), range(1, 20, 3))
    plt.xlim([-.1, 18.1])
    plt.savefig('2-example_dtw_sequences.pdf', bbox_inches='tight', pad_inches=0.1)

with sns.axes_style('white'):
    plt.figure(figsize=FIGSIZE)
    dist = scipy.spatial.distance.cdist(signal1.reshape(-1, 1), signal2.reshape(-1, 1))
    plt.imshow(dist, cmap=plt.cm.hot, interpolation='nearest')
    axis = plt.axis()
    for x, y in zip(q, p):
        plt.plot([x - .5, x - .5, x + .5, x + .5, x - .5], [y - .5, y + .5, y + .5, y - .5, y - .5], 'w')
    plt.axis(axis)
    plt.xticks(range(0, 19, 3), range(1, 20, 3))
    plt.yticks(range(0, 19, 3), range(1, 20, 3))
    plt.savefig('2-example_dtw_matrix.pdf', bbox_inches='tight', pad_inches=0.1)

# Chapter 3

In [ ]:
import cPickle as pickle
with open('data/3-statistics.pkl') as f:
    statistics = pickle.load(f)

In [ ]:
statistics = statistics[:10000]

In [ ]:
def uniform_hist(data, bins, ax, **kwargs):
    heights, _ = np.histogram(data, bins)
    ax.bar(left=np.arange(len(bins) - 1) - .5, height=heights,
           width=1, bottom=0, **kwargs)
    return heights

def pretty_hist(data, bins, fc, ax):
    """ Utility method for plotting a nice histogram """
    # Make it so that all points beyond the bin range get put in the last bin
    data = np.array(data)
    data[data > bins[-1]] = bins[-1] - 1e-10
    # Plot histogram, with specific coloring and axis-alignment
    heights = uniform_hist(data, bins, ax, fc=fc, alpha=.8)
    # Remove spines from plot
    sns.despine()
    # Add grid to y axis
    ax.yaxis.grid()
    # Set the plotting range to fit the histogram exactly
    bin_spacing = 1.
    ax.set_xlim(-bin_spacing/2., len(bins) - 1 - bin_spacing/2.)
    return heights

def divide_yticklabels(ax, divisor=1000):
    """ Utility method to scale down all y tick labels """
    ax.set_yticklabels([int(float(t)/divisor)
                        if (float(t)/divisor).is_integer()
                        else float(t)/divisor
                        for t in ax.get_yticks()])

def split_hist(data, bin_edges, high_bin_indices, fc):
    """ Plot a histogram where one or more bins have very large values """
    # Make high_bin_indices a list if an int was passed
    if isinstance(high_bin_indices, int):
        high_bin_indices = [high_bin_indices]
    # Create 2-row, 1-col subplot where the upper sublot is 1/4 the height
    # The upper subplot will be the tops of the very large bins; lower will be the rest
    gs = matplotlib.gridspec.GridSpec(2, 1, width_ratios=[1,], height_ratios=[1, 4])
    # Set the spacing between subplots to .1
    gs.update(hspace=0.1)
    # Grab axes handles
    ax = plt.subplot(gs[0])
    ax2 = plt.subplot(gs[1])
    # Plot pretty histograms both for the "upper" and "lower" parts of the split
    heights = pretty_hist(data, bin_edges, fc, ax)
    pretty_hist(data, bin_edges, fc, ax2)
    low_min = 0
    # Compute the height of the largest bin _not_ in high_bin_indices
    low_max = 1.1*max(heights[n] for n in range(len(bin_edges) - 1)
                      if n not in high_bin_indices)
    low_range = low_max - low_min
    # Compute the height of the smallest bin in high_bin_indices
    high_min = .9*min(heights[n] for n in high_bin_indices)
    # Compute the height of the highest bin in high_bin_indices
    high_max = 1.1*max(heights[n] for n in high_bin_indices)
    # Set the Y plotting range according to the above.  This will crop things.
    ax.set_ylim(high_min, high_max)
    ax2.set_ylim(low_min, low_max)
    # Hide the spines between ax and ax2
    ax.spines['bottom'].set_visible(False)
    ax2.spines['top'].set_visible(False)
    ax.xaxis.tick_top()
    ax.tick_params(labeltop='off')
    ax2.xaxis.tick_bottom()

    # Compute the spacing between y-ticks on the lower plot
    lowtick_spacing = np.diff(ax2.get_yticks())[0]
    # Create a single tick on the upper plot, rounded to the same spacing as lower plot
    ax.set_yticks([int(lowtick_spacing)*int((high_min + high_max)/(2*lowtick_spacing))])

    # X-axis start of clip lines (relative to [0, 1])
    start = -.015
    # Compute proportion of x-axis covered by last high_bin_indices (+ .015)
    end = (high_bin_indices[-1] + 1)/float(len(bin_edges) - 1) + .015
    # Plot the lines, allowing for it to expand outside of the axis
    ax.plot([start, end], [0., 0.], transform=ax.transAxes, color='k', clip_on=False)
    ax2.plot([start, end], [1., 1.], transform=ax2.transAxes, color='k', clip_on=False)

    # Convert count to thousands
    divide_yticklabels(ax)
    divide_yticklabels(ax2)

with sns.axes_style('white'):
    plt.figure(figsize=FIGSIZE)
    pretty_hist([s['n_instruments'] for s in statistics],
                range(22), BLUE, plt.gca())
    divide_yticklabels(plt.gca())
    plt.ylabel('Thousands of MIDI files')
    plt.xlabel('Number of instruments')
    plt.savefig('3-n_instruments.pdf', transparent=True, bbox_inches='tight', pad_inches=0.1)
    plt.xticks(range(0, 22, 5), range(0, 22 - 5, 5) + ['20+'])

    plt.figure(figsize=FIGSIZE)
    split_hist([i for s in statistics for i in s['program_numbers']],
               range(128), 0, BLUE)
    plt.ylabel('Thousands of occurrences')
    plt.xlabel('Program number')
    plt.savefig('3-program_numbers.pdf', transparent=True, bbox_inches='tight', pad_inches=0.1)


    plt.figure(figsize=FIGSIZE)
    split_hist([len(s['tempos']) for s in statistics],
               range(1, 12) + [30, 100, 1000], 0, GREEN)
    plt.xticks(range(13), range(1, 11) + ['11 - 30', '31 - 100', '101+'], rotation=45, ha='center')
    plt.ylabel('Thousands of MIDI files')
    plt.xlabel('Number of tempo changes', labelpad=-25)
    plt.savefig('3-n_tempos.pdf', transparent=True, bbox_inches='tight', pad_inches=0.1)

    plt.figure(figsize=FIGSIZE)
    pretty_hist([i for s in statistics for i in s['tempos']],
                range(0, 260, 10), GREEN, plt.gca())
    divide_yticklabels(plt.gca())
    plt.xticks(range(0, len(range(0, 260, 10)), 3), range(0, 240, 30) + ['240+'], rotation=45)
    plt.ylabel('Thousands of occurrences')
    plt.xlabel('Tempo', labelpad=-5)
    plt.savefig('3-tempos.pdf', transparent=True, bbox_inches='tight', pad_inches=0.1)

    plt.figure(figsize=FIGSIZE)
    split_hist([len(s['key_numbers']) for s in statistics],
               range(12), [0, 1], ORANGE)
    plt.xticks(range(11), range(10) + ['10+'])
    plt.ylabel('Thousands of MIDI files')
    plt.xlabel('Number of key changes')
    plt.savefig('3-n_keys.pdf', transparent=True, bbox_inches='tight', pad_inches=0.1)

    fig = plt.figure(figsize=FIGSIZE)
    split_hist([i for s in statistics for i in s['key_numbers']],
               range(25), 0, ORANGE)
    plt.xticks([0, 2, 4, 5, 7, 9, 11, 12, 14, 16, 17, 19, 21, 23],
               ['C', 'D', 'E', 'F', 'G', 'A', 'B', 'c', 'd', 'e', 'f', 'g', 'a', 'b'])
    plt.figtext(0.315, .07, 'Major', ha='center', va='center')
    plt.figtext(0.705, .07, 'Minor', ha='center', va='center')
    l1 = matplotlib.lines.Line2D([.14, .276], [.07, .07], c='k', transform=fig.transFigure, figure=fig)
    l2 = matplotlib.lines.Line2D([.35, .485], [.07, .07], c='k', transform=fig.transFigure, figure=fig)
    l3 = matplotlib.lines.Line2D([.54, .67], [.07, .07], c='k', transform=fig.transFigure, figure=fig)
    l4 = matplotlib.lines.Line2D([.74, .88], [.07, .07], c='k', transform=fig.transFigure, figure=fig)
    fig.lines.extend([l1, l2, l3, l4])
    plt.ylabel('Thousands of occurrences')
    plt.xlabel('Key', labelpad=1.5)
    plt.savefig('3-keys.pdf', transparent=True, bbox_inches='tight', pad_inches=0.1)

    plt.figure(figsize=FIGSIZE)
    split_hist([len(s['time_signature_changes']) for s in statistics],
               range(12), 1, RED)
    plt.xticks(range(11), range(10) + ['10+'])
    plt.ylabel('Thousands of MIDI files')
    plt.xlabel('Number of time signature changes')
    plt.savefig('3-n_signatures.pdf', transparent=True, bbox_inches='tight', pad_inches=0.1)

    # Get strings for all time signatures
    time_signatures = ['{}/{}'.format(c.numerator, c.denominator)
                       for s in statistics for c in s['time_signature_changes']]

    # Only display the n_top top time signatures
    n_top = 15
    # Get the n_top top time signatures
    top = collections.Counter(time_signatures).most_common(n_top)
    # Create a dict mapping an integer index to the time signature string
    top_signatures = {n: s[0] for n, s in enumerate(top)}
    # Add an additional index for non-top signatures
    top_signatures[n_top] = 'Other'
    # Compute the number of non-top time signatures
    n_other = len(time_signatures) - sum(s[1] for s in top)
    # Create a list with each index repeated the number of times
    # each time signature appears, to be passed to plt.hist
    indexed_time_signatures = sum([[n]*s[1] for n, s in enumerate(top)], [])
    indexed_time_signatures += [n_top]*n_other

    plt.figure(figsize=FIGSIZE)
    split_hist(indexed_time_signatures, range(n_top + 2), 0, RED)
    plt.xticks(top_signatures.keys(), top_signatures.values(), rotation=45, ha='center')
    plt.ylabel('Thousands of occurrences')
    plt.xlabel('Time signature', labelpad=-7)
    plt.savefig('3-time_signatures.pdf', transparent=True, bbox_inches='tight', pad_inches=0.1)

    pass

# Chapter 4

In [ ]:
ALIGNMENT_SEARCH_PATH = '/Users/craffel/Documents/projects/alignment-search/'
sys.path.append(ALIGNMENT_SEARCH_PATH)
import corrupt_midi
import create_data
import find_best_aligners
import db_utils

In [ ]:
# Some utility functions
def compute_cqt(audio_data):
    """ Compute the log-magnitude L2 normalized CQT """
    cqt, times = create_data.extract_cqt(audio_data)
    cqt = librosa.logamplitude(cqt, ref_power=cqt.max())
    return librosa.util.normalize(cqt, 2).T, times
def display_cqt(cqt):
    """ Plot a CQT with sane defaults """
    plt.imshow(cqt.T, aspect='auto', interpolation='nearest',
               origin='lower', cmap=plt.cm.hot,
               vmin=np.percentile(cqt, 1), vmax=np.percentile(cqt, 99))
    plt.yticks(range(0, 48, 12), [librosa.midi_to_note(n) for n in range(36, 36 + 48, 12)])

In [ ]:
np.random.seed(2)

# Grab a MIDI file from the clean MIDIs we used in this experiment
midi_file = os.path.join(ALIGNMENT_SEARCH_PATH, 'data/mid/Come as You Are.mid')
# Parse the MIDI file with pretty_midi
midi_object = pretty_midi.PrettyMIDI(midi_file)

with sns.axes_style('white'):
    # For illustration, we'll plot a CQT of the MIDI object
    # before and after corruptions.
    plt.figure(figsize=FIGSIZE_FLAT)
    original_cqt, original_times = compute_cqt(midi_object.fluidsynth(22050))
    display_cqt(original_cqt)
    #plt.title('Original MIDI CQT')
    plt.savefig('4-original_cqt.pdf', bbox_inches='tight', pad_inches=0.1)

# This is the wrapper function to apply all of the corruptions 
# defined in corrupt_midi
adjusted_times,  diagnostics = corrupt_midi.corrupt_midi(
    midi_object, original_times,
    # This defines the extent to which time will be warped
    warp_std=20,
    # These define how likely we are to crop out sections
    # We'll set them to 1 and 0 here for illustration; in the 
    # paper they are adjusted according to the desired corruption level
    start_crop_prob=0., end_crop_prob=0., middle_crop_prob=1.,
    # The likelihood that each instrument is removed
    remove_inst_prob=.5,
    # The likelihood that an instrument's program number is changed
    change_inst_prob=1.,
    # The standard deviation of velocity adjustment
    velocity_std=1.)

with sns.axes_style('white'):
    # Now, we can plot the CQT after corruptions.
    plt.figure(figsize=FIGSIZE_FLAT)
    corrupted_cqt, corrupted_times = compute_cqt(midi_object.fluidsynth(22050))
    display_cqt(corrupted_cqt)
    plt.xlabel('Frame')
    #plt.title('After corruption')
    plt.savefig('4-corrupted_cqt.pdf', bbox_inches='tight', pad_inches=0.1)

# We can also plot the timing offset, which we will try to reverse
plt.figure(figsize=FIGSIZE)
plt.plot(original_times, original_times - adjusted_times, BLUE, lw=2)
plt.xlim([0, original_times.max()])
plt.xlabel('Original time')
plt.ylabel('Offset from original time')
plt.savefig('4-warping.pdf', bbox_inches='tight', pad_inches=0.1)
sns.despine()

In [ ]:
# Compute a pairwise distance matrix of the original and corrupted CQTs
distance_matrix = scipy.spatial.distance.cdist(
    original_cqt, corrupted_cqt, 'sqeuclidean')
# Compute the lowest-cost path via DTW with "golden standard" parameters
p, q, score = djitw.dtw(
    distance_matrix, .96, np.median(distance_matrix), inplace=0)

# Compute the absolute error, clipped to within .5 seconds
plt.figure(figsize=FIGSIZE)
error = np.abs(np.clip(
    corrupted_times[q] - adjusted_times[p], -.5, .5))
plt.plot(original_times[p], error, BLUE, lw=2)
plt.xlabel('Time')
plt.ylabel('Correction error')
plt.xlim([0, original_times.max()])
plt.ylim([-0.01, .51])
plt.savefig('4-correction_error.pdf', bbox_inches='tight', pad_inches=0.1)

In [ ]:
# Load in the results from the parameter search experiment
params, objectives = db_utils.get_experiment_results(
    os.path.join(ALIGNMENT_SEARCH_PATH, 'results/parameter_experiment_gp/*.json'))
# Truncate to the top 20 results
good = np.argsort(objectives)[:10]
params = [params[n] for n in good]
objectives = [objectives[n] for n in good]
# Pretty-print using tabulate
for param, objective in zip(params, objectives):
    param['objective'] = objective
header_names = collections.OrderedDict([
    ('add_pen', '$\phi$ Median Scale'),
    ('standardize', 'Standardize?'),
    ('gully', 'Gully $g$'),
    ('objective', 'Mean Error')])
def yes_no(x):
    if isinstance(x, bool):
        if x:
            return 'Yes'
        else:
            return 'No'
    else:
        return x
print tabulate.tabulate([collections.OrderedDict([(k, yes_no(p[k])) for k in header_names]) for p in params],
                        headers=header_names, tablefmt='latex_booktabs')

In [ ]:
# Load in all confidence reporting experiment trials
trials = []
for trial_file in glob.glob(os.path.join(ALIGNMENT_SEARCH_PATH, 'results/confidence_experiment/*.json')):
    with open(trial_file) as f:
        trials.append(json.load(f))
# Retrieve the lowest-achieved mean absolute error
best_easy_error = objectives[0]
# Retrieve the confidence reporting trial for this system
best_trial = [t for t in trials
               if np.allclose(np.mean(t['results']['easy_errors']),
                              best_easy_error)][0]
# Retrieve the results from this trial
best_result = best_trial['results']

# Plot a scatter plot of mean alignment error vs. confidence score
errors = np.array(best_result['hard_errors'] + best_result['easy_errors'])
scores = np.array(best_result['hard_penalty_len_norm_mean_norm_scores'] +
                  best_result['easy_penalty_len_norm_mean_norm_scores'])
plt.figure(figsize=FIGSIZE)
plt.scatter(errors, scores, marker='+', c='black', alpha=.3, s=40)
plt.gca().set_xscale('log')
plt.ylim(0., 1.1)
plt.xlim(.9*np.min(errors), np.max(errors)*1.1)
plt.xlabel('Alignment error')
plt.ylabel('Normalized DTW distance')
plt.xticks([.01, .025, .05, .1, .25, .5], [.01, .025, .05, .1, .25, .5])
plt.savefig('4-correlation.pdf', bbox_inches='tight', pad_inches=0.1)

In [ ]:
with open(os.path.join(ALIGNMENT_SEARCH_PATH, 'results/alignment_ratings.csv')) as f:
    reader = csv.reader(f)
    # Cast each entry in each row to the correct type
    ratings = [[int(alignment_id), int(rating), np.clip(2*(1 - float(score)), 0, 1), note]
               for alignment_id, rating, score, note in reader]

# We made notes about each alignment, too.
# Here are all the alignments where a transcription was matched to a remix
remixes = [r[1:] for r in ratings if ('remix' in r[-1].lower())]
remixes.sort(key = lambda x: -x[1])
print tabulate.tabulate(remixes, headers=['Rating', 'Confidence Score', 'Note'], tablefmt='latex_booktabs')

In [ ]:
# Plot a histogram for each rating
plt.figure(figsize=FIGSIZE)
data = [np.array([r[2] for r in ratings if r[1] == n]) for n in [1, 2, 3, 4, 5]]
violins = plt.violinplot(
    data, showextrema=False, showmeans=False,
    widths=[float(len(d))/max(len(d) for d in data) for d in data])
patches = plt.boxplot(data, showmeans=False, showcaps=False, showfliers=False, 
                      patch_artist=True, widths=.1)
for line in patches['whiskers']:
    line.set_visible(False)
for box in patches['boxes']:
    box.set_facecolor('None')
    box.set_alpha(.5)
    box.set_joinstyle('round')
    box.set_facecolor('w')
    box.set_edgecolor('k')
for line in patches['medians']:
    line.set_color('black')
for body in violins['bodies']:
    body.set_alpha(.8)
for n in [0, 1]:
    violins['bodies'][n].set_facecolor(BLUE)
for n in [2, 3, 4]:
    violins['bodies'][n].set_facecolor(GREEN)
plt.xticks(
    [1, 2, 3, 4, 5],
    [1, 2, 3, 4, 5])
    #['Wrong song', 'Bad alignment', 'Sloppy', 'Embellishments', 'Perfect'],
    #rotation=20)
plt.xlim([.5, 5.5])
plt.xlabel('Rating')
plt.ylabel('Confidence score')
plt.legend(handles=[matplotlib.patches.Patch(color=BLUE, label='Incorrect'),
                    matplotlib.patches.Patch(color=GREEN, label='Correct')],
           loc='upper left')
plt.ylim(-.03, 1.03)
plt.savefig('4-violin.pdf',  bbox_inches='tight', pad_inches=0.1)

## Chapter 5

In [ ]:
def pretty_cqt(audio_data, fs=feature_extraction.AUDIO_FS):
    gram = np.abs(librosa.cqt(
        audio_data, sr=fs, hop_length=feature_extraction.AUDIO_HOP,
        fmin=librosa.midi_to_hz(feature_extraction.NOTE_START),
        n_bins=feature_extraction.N_NOTES, real=False))
    # Compute log amplitude
    gram = librosa.logamplitude(gram**2, ref_power=gram.max())
    # Transpose so that rows are samples
    gram = gram.T
    # and L2 normalize
    #gram = librosa.util.normalize(gram, axis=1)
    # and convert to float32
    return gram.astype(np.float32)
audio, fs = librosa.load('data/5-mmt.wav', sr=feature_extraction.AUDIO_FS)
audio_gram = pretty_cqt(audio)
m = pretty_midi.PrettyMIDI('data/5-mmt.mid')
midi_audio_aligned = m.fluidsynth(feature_extraction.AUDIO_FS)
# Adjust to the same size as audio
if midi_audio_aligned.shape[0] > audio.shape[0]:
    midi_audio_aligned = midi_audio_aligned[:audio.shape[0]]
else:
    trim_amount = audio.shape[0] - midi_audio_aligned.shape[0]
    midi_audio_aligned = np.append(midi_audio_aligned,
                                   np.zeros(trim_amount))
midi_aligned_gram = pretty_cqt(midi_audio_aligned)

In [ ]:
def draw_brace(ax, left, right, bottom, top, beta=10., fliplr=False, flipud=False):
    if flipud:
        bottom, left = left, bottom
        right, top = top, right
    half_y = (top + bottom)/2.
    half_range = np.linspace(bottom, half_y, 100)
    x = (1/(1. + np.exp(beta*(half_range - bottom)))
         + 1/(1. + np.exp(beta*(half_range - half_y))))
    x = np.concatenate((x, x[::-1]))
    if fliplr:
        x = -x + 1
    else:
        x = x - 1
    x = x*(right - left) + (left + right)/2.
    if flipud:
        ax.plot(np.linspace(bottom, top, 200), x, 'k', lw=2)
    else:
        ax.plot(x, np.linspace(bottom, top, 200), 'k', lw=2)

In [ ]:
fig = plt.figure(figsize=(FIGSIZE[0], FIGSIZE[0]))
ax = plt.gca()
draw_neural_net(ax, .3, .6, .51, .79, [4, 7, 5, 3])
ax.text(.32, .76, '$f$', ha='center', va='center', fontdict={'size': 30})
draw_neural_net(ax, .3, .6, .21, .49, [4, 6, 7, 3])
ax.text(.32, .46, '$g$', ha='center', va='center', fontdict={'size': 30})

ax.annotate('', xy=(.15, .8),  xycoords='data',
            xytext=(.25, .65), textcoords=None,
            arrowprops=dict(arrowstyle="<-",
                            connectionstyle="angle,angleA=0,angleB=90,rad=10",
                            lw=2),
            size=40)
draw_brace(ax, .255, .285, .58, .72, 1000.)

ax.annotate('', xy=(.15, .2),  xycoords='data',
            xytext=(.25, .35), textcoords=None,
            arrowprops=dict(arrowstyle="<-",
                            connectionstyle="angle,angleA=0,angleB=90,rad=10",
                            lw=2),
            size=40)
draw_brace(ax, .255, .285, .28, .42, 1000.)

ax.annotate('', xy=(.65, .65),  xycoords='data',
            xytext=(.8, .55), textcoords=None,
            arrowprops=dict(arrowstyle="<-",
                            connectionstyle="angle,angleA=90,angleB=0,rad=10",
                            lw=2),
            size=40)
draw_brace(ax, .615, .645, .6, .7, 1000., 1)

ax.annotate('', xy=(.65, .35),  xycoords='data',
            xytext=(.8, .45), textcoords=None,
            arrowprops=dict(arrowstyle="<-",
                            connectionstyle="angle,angleA=90,angleB=0,rad=10",
                            lw=2),
            size=40)
draw_brace(ax, .615, .645, .3, .4, 1000., 1)

ax.imshow(audio_gram[500:700].T, interpolation='nearest', aspect='auto', cmap=plt.cm.hot,
          origin='lower', extent=(0, 1, .8, 1), vmin=np.percentile(audio_gram, 15))
ax.imshow(midi_aligned_gram[500:700].T, interpolation='nearest', aspect='auto', cmap=plt.cm.hot,
          origin='lower', extent=(0, 1, 0, .2), vmin=np.percentile(midi_aligned_gram, 15))

ax.imshow(np.array([[1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0]]), cmap=plt.cm.hot,
          extent=(.6, 1., .46, .485), interpolation='nearest')

ax.imshow(np.array([[1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0]]), cmap=plt.cm.hot,
          extent=(.6, 1., .51, .535), interpolation='nearest')

ax.add_patch(matplotlib.patches.Rectangle((0.13, 0.001), 0.04, 0.2, alpha=.6, fc='w'))
ax.add_patch(matplotlib.patches.Rectangle((0.13, 0.8), 0.04, 0.2, alpha=.6, fc='w'))

ax.add_patch(matplotlib.patches.Rectangle((0.6, 0.46), 0.4, 0.025, fc='None', ec='k'))
ax.add_patch(matplotlib.patches.Rectangle((0.6, 0.51), 0.4, 0.025, fc='None', ec='k'))

ax.axis([0, 1, 0, 1])
ax.axis('off')
plt.savefig('5-hashing_schematic.pdf',  bbox_inches='tight', pad_inches=0.1)

In [ ]:
hash_a = np.array([1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0], dtype=bool)
hash_b = np.array([1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0], dtype=bool)
xor = np.bitwise_xor(hash_a, hash_b)
popcnt = np.sum(xor)

plt.figure(figsize=FIGSIZE)
eps = .002
ax = plt.gca()

hash_l = .24
hash_r = .995
vheight = (hash_l - hash_r)/16

plt.imshow(hash_a[np.newaxis], interpolation='nearest',
           extent=(hash_l, hash_r, 1., 1 - vheight + eps), cmap=plt.cm.hot)
ax.add_patch(plt.Rectangle((hash_l, 1 - vheight), hash_r - hash_l, vheight, fc='none'))

plt.imshow(hash_b[np.newaxis], interpolation='nearest',
           extent=(hash_l, hash_r, 1 - 3*vheight/2 + eps, 1 - 5*vheight/2), cmap=plt.cm.hot)
ax.add_patch(plt.Rectangle((hash_l, 1 - 5*vheight/2), hash_r - hash_l, vheight, fc='none'))

plt.imshow(xor[np.newaxis], interpolation='nearest',
           extent=(hash_l, hash_r, 1 - 3*vheight, 1 - 4*vheight), cmap=plt.cm.hot)
ax.add_patch(plt.Rectangle((hash_l, 1 - 4*vheight), hash_r - hash_l, vheight, fc='none'))

plt.plot([hash_l*.7, hash_r], [1 - 11*vheight/4, 1 - 11*vheight/4], 'k', lw=2)

ax.add_patch(plt.Circle((hash_l*.8, 1 - 2*vheight), vheight/2, fc='none', lw=2))
plt.plot((hash_l*.8, hash_l*.8), (1 - 3*vheight/2, 1 - 5*vheight/2), 'k', lw=2)
plt.plot((hash_l*.8 - vheight/2, hash_l*.8 + vheight/2), (1 - 2*vheight, 1 - 2*vheight), 'k', lw=2)

plt.text(hash_l/2, 1 - 7*vheight/2 + .003, 'POPCNT(', va='center', ha='center',
         fontdict={'size': 28, 'family': 'monospace'})
plt.text(hash_r + (1 - hash_r)/2., 1 - 7*vheight/2 + .003, ')$\;$=$\;${}'.format(popcnt),
         va='center', ha='left', fontdict={'size': 28, 'family': 'monospace'})

plt.axis([0, 1, 1 - 4*vheight + .01, 1])
plt.axis('off')

plt.savefig('5-popcnt.pdf',  bbox_inches='tight', pad_inches=0.1)

In [ ]:
# How many false positives/negatives do we get from thresholding the alignment scores at .5?
with open(os.path.join(ALIGNMENT_SEARCH_PATH, 'results/alignment_ratings.csv')) as f:
    reader = csv.reader(f)
    # Cast each entry in each row to the correct type
    ratings = [[int(alignment_id), int(rating), np.clip(2*(1 - float(score)), 0, 1), note]
               for alignment_id, rating, score, note in reader]
print np.sum([r[1] <= 2 and r[2] >= .5 for r in ratings])
print np.sum([r[1] >= 3 and r[2] <= .5 for r in ratings])
print np.sum([r[1] >= 3 and r[2] >= .5 for r in ratings])

In [ ]:
import lasagne
import theano
import deepdish
import glob
import sys
sys.path.append('/Users/craffel/Documents/projects/midi-dataset/')
import experiment_utils
import whoosh_search
import feature_extraction
RESULTS_PATH = '/Users/craffel/Documents/projects/midi-dataset/results/'
import dhs

In [ ]:
def paa(sequence, window):
    """
    Compute ``Piecewise Aggregate Approximation'' of a sequence, which simply
    computes the local mean of the sequence over non-overlapping windows
    Parameters
    ----------
    sequence : np.ndarray
        A sequence; will be aggregated over its first dimension.
    window : int
        Size of windows over which to compute the mean
    Returns
    -------
    aggregated_sequence : np.ndarray
        Aggregated sequence, the first dimension will be 1/window of its
        original size
    """
    # Truncate sequence to rounded-down nearest divisor
    sequence = sequence[:window*int(sequence.shape[0]/window)]
    # Reshape sequence to be of shape
    # (original_shape[0]/window, window, original trailing dimensions...)
    sequence = sequence.reshape(
            sequence.shape[0]/window, window, sequence.shape[-1])
    # Compute mean over the window dimension
    return sequence.mean(axis=1)

In [ ]:
# Load in all parameter optimization trials
trial_files = glob.glob(os.path.join(
    RESULTS_PATH, 'dhs_parameter_trials', '*.h5'))
trials = [deepdish.io.load(f) for f in trial_files]
# Get the hyperparameters for the trial with the lowest objective value
best_trial = sorted(trials, key=lambda t: t['best_objective'])[0]
hyperparameters = best_trial['hyperparameters']
# Load in the pre-trained parameters for the best performing models
network_params = deepdish.io.load(
    os.path.join(RESULTS_PATH, 'dhs_model', 'best_model.h5'))
compute_output = {}
training_mean = {}
# Build networks and output-computing functions
for dataset, network in zip(['clean_midi', 'msd'], ['X', 'Y']):
    # Construct the network according to best-trial hyperparameters
    if hyperparameters['network'] == 'dhs_big_filter':
        build_network = experiment_utils.build_dhs_net_big_filter
    elif hyperparameters['network'] == 'dhs_small_filters':
        build_network = experiment_utils.build_dhs_net_small_filters
    layers = build_network(
        (None, 1, None, feature_extraction.N_NOTES),
        # We will supply placeholders here but load in the values below
        np.zeros((1, feature_extraction.N_NOTES), theano.config.floatX),
        np.ones((1, feature_extraction.N_NOTES), theano.config.floatX),
        hyperparameters['downsample_frequency'])
    # Load in network parameter values
    lasagne.layers.set_all_param_values(
        layers[-1], network_params[network])
    # Retrieve the training set mean from the bias layer created by the standardization layer
    training_mean[network] = -lasagne.layers.get_all_layers(layers[-1])[1].b.get_value()
    # Compile function for computing the output of the network
    compute_output[network] = theano.function(
        [layers[0].input_var],
        lasagne.layers.get_output(layers[-1], deterministic=True))

In [ ]:
# Load in all parameter optimization trials
trial_files = glob.glob(os.path.join(
    RESULTS_PATH, 'dhs_piano_parameter_trials', '*.h5'))
trials = [deepdish.io.load(f) for f in trial_files]
# Get the hyperparameters for the trial with the lowest objective value
best_trial = sorted(trials, key=lambda t: t['best_objective'])[0]
hyperparameters = best_trial['hyperparameters']
# Load in the pre-trained parameters for the best performing models
network_params = deepdish.io.load(
    os.path.join(RESULTS_PATH, 'dhs_piano_model', 'best_model.h5'))
compute_output_piano = {}
# Build networks and output-computing functions
for dataset, network in zip(['clean_midi', 'msd'], ['X', 'Y']):
    # Construct the network according to best-trial hyperparameters
    if hyperparameters['network'] == 'dhs_big_filter':
        build_network = experiment_utils.build_dhs_net_big_filter
    elif hyperparameters['network'] == 'dhs_small_filters':
        build_network = experiment_utils.build_dhs_net_small_filters
    layers = build_network(
        (None, 1, None, feature_extraction.N_NOTES),
        # We will supply placeholders here but load in the values below
        np.zeros((1, feature_extraction.N_NOTES), theano.config.floatX),
        np.ones((1, feature_extraction.N_NOTES), theano.config.floatX),
        hyperparameters['downsample_frequency'])
    # Load in network parameter values
    lasagne.layers.set_all_param_values(
        layers[-1], network_params[network])
    # Compile function for computing the output of the network
    compute_output_piano[network] = theano.function(
        [layers[0].input_var],
        lasagne.layers.get_output(layers[-1], deterministic=True))

In [ ]:
n_examples = 100

validation_path = os.path.join(RESULTS_PATH, 'training_dataset', 'validate', 'h5')
validation_raw = [deepdish.io.load(f) for f in glob.glob(os.path.join(validation_path, '*.h5'))[:n_examples]]
validation_raw = [dict((k, d[k][0]) for k in ['X', 'Y']) for d in validation_raw]

validation_path = os.path.join(RESULTS_PATH, 'training_dataset_piano_roll', 'validate', 'h5')
validation_piano = [deepdish.io.load(f) for f in glob.glob(os.path.join(validation_path, '*.h5'))[:n_examples]]
validation_piano = [dict((k, d[k][0]) for k in ['X', 'Y']) for d in validation_piano]

In [ ]:
validation_out = [dict((k, compute_output[k](d[k].reshape(1, 1, *d[k].shape))) for k in ['X', 'Y'])
                  for d in validation_raw]
validation_out_piano = [dict((k, compute_output_piano[k](d[k].reshape(1, 1, *d[k].shape))) for k in ['X', 'Y'])
                        for d in validation_piano]
validation_paa = [dict((k, paa(d[k], 8)) for k in ['X', 'Y']) for d in validation_raw]

In [ ]:
validation_hash = [dict((k, 1*(d[k] > 0)) for k in ['X', 'Y']) for d in validation_out]
validation_hash_piano = [dict((k, 1*(d[k] > 0)) for k in ['X', 'Y']) for d in validation_out_piano]
validation_tpaa = [dict((k, 1*(d[k] > training_mean[k])) for k in ['X', 'Y']) for d in validation_paa]

In [ ]:
representations = [validation_raw, validation_hash, 
                   validation_piano, validation_hash_piano,
                   validation_out, validation_tpaa]
fig = plt.figure(figsize=(FIGSIZE[0], 1.5*FIGSIZE[1]))
for n, rep in enumerate(representations):
    all_X = np.concatenate([d['X'] for d in rep])
    all_Y = np.concatenate([d['Y'] for d in rep])
    all_Y_n = all_Y[dhs.random_derangement(all_Y.shape[0])]
    p_distances = np.sum((all_X - all_Y)**2, axis=1)
    n_distances = np.sum((all_X - all_Y_n)**2, axis=1)
    if rep is validation_hash or rep is validation_hash_piano or rep is validation_tpaa:
        min_dist = 0
        max_dist = 32
    elif rep is validation_piano:
        min_dist = 1.9
        max_dist = max(max(p_distances), max(n_distances))
    else:
        min_dist = 0
        max_dist = max(np.percentile(p_distances, 99), np.percentile(n_distances, 99))
    if n % 2:
        colors = [BLUE, GREEN]
    else:
        colors = [RED, ORANGE]
    plt.subplot(3, 2, n + 1)
    plt.hist(p_distances, 32, range=[min_dist, max_dist], fc=colors[0], alpha=.7,
             weights=np.ones(p_distances.shape[0])/p_distances.shape[0])
    plt.hist(n_distances, 32, range=[min_dist, max_dist], fc=colors[1], alpha=.7,
             weights=np.ones(n_distances.shape[0])/n_distances.shape[0])
    plt.xlim(min_dist, max_dist)
    pos, labels = plt.yticks()
    if np.diff(pos)[0] < .02:
        plt.yticks(np.linspace(0, .1, 6, endpoint=True))

fig.text(0.5, 0.085, 'Distance', ha='center', va='center')
fig.text(0.065, 0.5, 'Proportion', ha='center', va='center', rotation='vertical')
        
#plt.tight_layout()
plt.savefig('5-distributions.pdf',  bbox_inches='tight', pad_inches=0.1)

In [ ]:
# Get pre-computed CQT for matching MIDI and MSD
midi_gram = deepdish.io.load('data/5-midi-features.h5')['gram']
audio_gram = deepdish.io.load('data/5-msd-features.h5')['gram']
# Construct PrettyMIDI object to get piano roll
pm = pretty_midi.PrettyMIDI('data/5-midi.mid')
# Create list of frames whose spacing matches CQT spacing
max_frame = int(pm.get_end_time() *
                feature_extraction.MIDI_FS /
                feature_extraction.MIDI_HOP)
midi_frame_times = librosa.frames_to_time(
    np.arange(max_frame), sr=feature_extraction.MIDI_FS,
    hop_length=feature_extraction.MIDI_HOP)
# Retrieve piano roll
piano_roll = pm.get_piano_roll(times=midi_frame_times)
# Only utilize the same notes which are used in the CQT
piano_roll = piano_roll[
    feature_extraction.NOTE_START:
    (feature_extraction.NOTE_START +
     feature_extraction.N_NOTES)]
piano_roll = piano_roll.T
piano_roll = librosa.util.normalize(piano_roll, norm=2, axis=1)
# Use float32 for Theano
piano_roll = piano_roll.astype(np.float32)

midi_out = compute_output['X'](midi_gram.reshape(1, 1, *midi_gram.shape))
msd_out = compute_output['Y'](audio_gram.reshape(1, 1, *audio_gram.shape))
midi_hash_vectors = midi_out > 0
msd_hash_vectors = msd_out > 0

midi_tpaa = paa(midi_gram, 8) > training_mean['X']
msd_tpaa = paa(audio_gram, 8) > training_mean['Y']

midi_piano_hash_vectors = compute_output_piano['X'](piano_roll.reshape(1, 1, *piano_roll.shape)) > 0
msd_piano_hash_vectors = compute_output_piano['Y'](audio_gram.reshape(1, 1, *audio_gram.shape)) > 0

with sns.axes_style('white'):
    fig = plt.figure(figsize=FIGSIZE_FLAT)
    gs1 = matplotlib.gridspec.GridSpec(
        2, 2, width_ratios=[audio_gram.shape[0],
                            midi_gram.shape[0]],
        height_ratios=[48, 32])
    gs1.update(top=1., bottom=0., wspace=.04, hspace=.35)

    cqt_spacing = 400

    ax = plt.subplot(gs1[0])
    ax.imshow(audio_gram.T, aspect='auto', origin='lower',
              interpolation='nearest', cmap=plt.cm.hot,
              vmin=np.percentile(audio_gram, 1),
              vmax=np.percentile(audio_gram, 99))
    ax.set_xticks(np.arange(0, audio_gram.shape[0], cqt_spacing))
    ax.set_yticks(np.arange(0, audio_gram.shape[1], 12))
    ax.set_yticklabels(['C{}'.format(n) for n in [3, 4, 5, 6]])
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')

    ax = plt.subplot(gs1[1])
    ax.imshow(midi_gram.T, aspect='auto', origin='lower',
              interpolation='nearest', cmap=plt.cm.hot,
              vmin=np.percentile(midi_gram, 1),
              vmax=np.percentile(midi_gram, 99))
    ax.set_xticks(np.arange(0, midi_gram.shape[0], cqt_spacing))
    ax.set_yticks(np.arange(0, audio_gram.shape[1], 12))
    ax.set_yticklabels(['' for _ in [3, 4, 5, 6]])
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')

    hash_spacing = cqt_spacing/8

    ax = plt.subplot(gs1[2])
    ax.imshow(msd_hash_vectors.T, aspect='auto', origin='lower',
              interpolation='nearest', cmap=plt.cm.hot)
    ax.set_xticks(np.arange(0, msd_hash_vectors.shape[0], hash_spacing))
    ax.set_yticks(np.arange(0, msd_hash_vectors.shape[1], 8))
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')

    ax = plt.subplot(gs1[3])
    ax.imshow(midi_hash_vectors.T, aspect='auto', origin='lower',
              interpolation='nearest', cmap=plt.cm.hot)
    ax.set_xticks(np.arange(0, midi_hash_vectors.shape[0], hash_spacing))
    ax.set_yticks(np.arange(0, midi_hash_vectors.shape[1], 8))
    ax.set_yticklabels(['' for _ in np.arange(0, msd_hash_vectors.shape[1], 4)])
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    fig.text(0.5, -0.15, 'Sequence Index', ha='center', va='center')
    fig.text(0.085, 0.5, 'Feature', ha='center', va='center', rotation='vertical')
    plt.savefig('5-cqts_and_dhs.pdf',  bbox_inches='tight', pad_inches=0.1)    

    
    
    cqt_spacing_x = 400
    cqt_spacing_y = 200
    hash_spacing_x = cqt_spacing_x/8
    hash_spacing_y = cqt_spacing_y/8

    fig = plt.figure(figsize=(FIGSIZE[0], 8))
    ax = plt.subplot(4, 1, 1)
    dist_mat = (1 - np.dot(midi_gram, audio_gram.T))
    ax.imshow(dist_mat.T,
              interpolation='nearest', cmap=plt.cm.gray,
              vmin=np.percentile(dist_mat, 1),
              vmax=np.percentile(dist_mat, 99.9))
    tight = ax.axis()
    p, q, _ = djitw.dtw(dist_mat, .96, np.median(dist_mat), inplace=False)
    ax.plot(p, q, 'w--', lw=3)
    ax.axis(tight)
    ax.set_xticks(np.arange(0, midi_gram.shape[0], cqt_spacing_x))
    ax.set_yticks(np.arange(0, audio_gram.shape[0], cqt_spacing_y))

    dist_mat = scipy.spatial.distance.cdist(midi_hash_vectors, msd_hash_vectors, 'sqeuclidean')
    ax = plt.subplot(4, 1, 2)
    ax.imshow(dist_mat.T, interpolation='nearest', cmap=plt.cm.gray)
    tight = ax.axis()
    p, q, _ = djitw.dtw(dist_mat, .9, 15, inplace=False)
    ax.axis(tight)
    ax.plot(p, q, 'w--', lw=3)
    ax.set_xticks(np.arange(0, midi_hash_vectors.shape[0], hash_spacing_x))
    ax.set_yticks(np.arange(0, msd_hash_vectors.shape[0], hash_spacing_y))

    dist_mat = scipy.spatial.distance.cdist(midi_piano_hash_vectors, msd_piano_hash_vectors, 'sqeuclidean')
    ax = plt.subplot(4, 1, 3)
    ax.imshow(dist_mat.T, interpolation='nearest', cmap=plt.cm.gray)
    tight = ax.axis()
    p, q, _ = djitw.dtw(dist_mat, .9, 15, inplace=False)
    ax.axis(tight)
    ax.plot(p, q, 'w--', lw=3)
    ax.set_xticks(np.arange(0, midi_piano_hash_vectors.shape[0], hash_spacing_x))
    ax.set_yticks(np.arange(0, msd_piano_hash_vectors.shape[0], hash_spacing_y))

    dist_mat = scipy.spatial.distance.cdist(midi_tpaa, msd_tpaa, 'sqeuclidean')
    ax = plt.subplot(4, 1, 4)
    ax.imshow(dist_mat.T, interpolation='nearest', cmap=plt.cm.gray)
    tight = ax.axis()
    p, q, _ = djitw.dtw(dist_mat, .9, 15, inplace=False)
    ax.axis(tight)
    ax.plot(p, q, 'w--', lw=3)
    ax.set_xticks(np.arange(0, midi_tpaa.shape[0], hash_spacing_x))
    ax.set_yticks(np.arange(0, msd_tpaa.shape[0], hash_spacing_y))
    
    fig.text(0.5, 0.085, 'MIDI Sequence Index', ha='center', va='center')
    fig.text(0.085, 0.5, 'Audio Sequence Index', ha='center', va='center', rotation='vertical')
    plt.savefig('5-distance_matrices.pdf',  bbox_inches='tight', pad_inches=0.1)    

In [ ]:
def prop_below(results):
    ranks = [min(r['msd_match_ranks']) for r in results]
    lt = np.less.outer(ranks, np.arange(1000000))
    return np.mean(lt, axis=0)

In [ ]:
rank_curves = {}
for rep in ['dhs', 'dhs_piano', 'tpaa']:
    results = deepdish.io.load(os.path.join(RESULTS_PATH, '{}_match_results/test_results.h5'.format(rep)))
    rank_curves[rep] = prop_below(results)
    ranks = [min(r['msd_match_ranks']) for r in results]
    print '{} MRR: {}, rank 1: {}, rank for < 95%: {}'.format(
        rep,
        np.mean([1./(r + 1) for r in ranks]),
        np.mean(np.less(ranks, 1)),
        np.argmax(rank_curves[rep] > .95) + 1)

In [ ]:
plt.figure(figsize=FIGSIZE)
for rep, color in zip(['dhs', 'dhs_piano', 'tpaa'], [BLUE, GREEN, RED]):
    plt.semilogx(100*rank_curves[rep], color, lw=2)
plt.ylim(0, 102)
plt.ylabel('Percentage below')
plt.xlabel('Rank')
plt.text(1.2, 87, 'DHS, CQT', {'color': BLUE, 'weight': 'bold', 'size': 16})
plt.text(10, 76, 'DHS, Piano roll', {'color': GREEN, 'weight': 'bold', 'size': 16})
plt.text(10, 76, 'DHS, Piano roll', {'color': 'k', 'weight': 'bold', 'size': 16}, alpha=.1)
plt.text(100, 53, 'TPAA', {'color': RED, 'weight': 'bold', 'size': 16})
plt.savefig('5-ranks.pdf',  bbox_inches='tight', pad_inches=0.1)    

## Chapter 6

In [ ]:
canvas_height = .7

plt.figure(figsize=(FIGSIZE[0], FIGSIZE[0]*canvas_height))

eps = 1e-3

ax = plt.gca()
axis = ax.axis((0, 1, 0, canvas_height))

# Grid
if 0:
    for n in np.linspace(0, 1, 21):
        plt.plot([0, 1], [n, n], 'k:')
        plt.plot([n, n], [0, 1], 'k:')
        ax.annotate(s=str(n),
                xy=(0., n), ha='center', va='center', family='sans-serif')
        ax.annotate(s=str(n),
                xy=(n, 1.), ha='center', va='center', family='sans-serif')


# h_t boxes
h_t_width = .1
h_t_height = .3
sum_bottom = .53
for m, n in enumerate([0, 1, 2, 4]):
    h_box = matplotlib.patches.Rectangle(
        (.05 + n/5., eps), h_t_width, h_t_height,
        facecolor=GREEN, edgecolor='k', lw=2)
    ax.add_patch(h_box)

    plt.plot([.05 + n/5. + h_t_width/2., .5],
             [h_t_height, sum_bottom], '#cccccc', lw=2, zorder=-2)

    circle_center_x = (.05 + n/5. + h_t_width/2. + .5)/2.
    circle_center_y = (h_t_height + sum_bottom)/2.
    circle_radius = .02
    circle = plt.Circle((circle_center_x, circle_center_y), circle_radius,
                        color=ORANGE, ec='k', zorder=4)
    ax.add_artist(circle)
    circle_offset = .8*(np.sqrt(2)/2)*circle_radius
    plt.plot([circle_center_x - circle_offset, circle_center_x + circle_offset],
             [circle_center_y - circle_offset, circle_center_y + circle_offset],
             'k', lw=1.5, zorder=5)
    plt.plot([circle_center_x - circle_offset, circle_center_x + circle_offset],
             [circle_center_y + circle_offset, circle_center_y - circle_offset],
             'k', lw=1.5, zorder=5)
    
    spline_y = [.36, .38, .39*.1 + .9*circle_center_x, circle_center_x][::-1]
    spline_x = [.55, .54 + .01*n/6., circle_center_y + circle_radius*2,
                circle_center_y + circle_radius][::-1]
    interpolated_x = np.linspace(spline_x[0], spline_x[-1], 100)
    interpolated_y = scipy.interpolate.spline(spline_x, spline_y, interpolated_x, 3)
    plt.plot(interpolated_y, interpolated_x, 'k', ls='dashed')

    spline_y = [.06, .05 - .03*m/4., .05 + .2*n/5., .05 + n/5. + h_t_width/2][::-1]
    spline_x = [.55, .5 + .02*m/4., .4, h_t_height][::-1]
    interpolated_x = np.linspace(spline_x[0], spline_x[-1], 100)
    interpolated_y = scipy.interpolate.spline(spline_x, spline_y, interpolated_x, 3)
    plt.plot(interpolated_y, interpolated_x, 'k', ls='dashed')
    
    ax.text(circle_center_x + .015*(n == 0), circle_center_y + circle_offset,
            '$\\alpha[{}]$'.format(n + 1 if n < 3 else 'T\,'),
            horizontalalignment='left',
            verticalalignment='bottom', fontsize=22, zorder=5)
    
    ax.text(.05 + n/5. + h_t_width/2, h_t_height/2, '$h[{}]$'.format(n + 1 if n < 3 else 'T\,'),
            horizontalalignment='center',
            verticalalignment='center', fontsize=26, zorder=5)
    
    
    
# Dots
dot_spacing = .05
for offset in [-1, 0, 1]:
    circle = plt.Circle((.7 + offset*dot_spacing, h_t_height/2),
                        .007, color='k', ec='k', zorder=4)
    ax.add_artist(circle)

draw_neural_net(ax, .1, .35, .4, .7, [3, 5, 1])
draw_brace(ax, .06, .08, .475, .625, 1000., 0, 0)

ax.text(.1, .7, '$a(h[t])$',
        horizontalalignment='center',
        verticalalignment='top', fontsize=24, zorder=5)
    
# + circle
circle_center_x = .5
circle_center_y = .55
circle_radius = .02
circle = plt.Circle((circle_center_x, circle_center_y), circle_radius,
                     color=ORANGE, ec='k', zorder=4)
ax.add_artist(circle)
circle_offset = .8*circle_radius
plt.plot([circle_center_x, circle_center_x],
         [circle_center_y - circle_offset, circle_center_y + circle_offset],
         'k', lw=1.5, zorder=5)
plt.plot([circle_center_x - circle_offset, circle_center_x + circle_offset],
         [circle_center_y, circle_center_y],
         'k', lw=1.5, zorder=5)

c_box = matplotlib.patches.Rectangle(
    (.85 - h_t_width/2., .55 - h_t_height/2. - eps), h_t_width, h_t_height,
    facecolor=BLUE, edgecolor='k', lw=2, alpha=.7)
ax.add_patch(c_box)
ax.text(.85, .55, '$c$',
        horizontalalignment='center',
        verticalalignment='center', fontsize=30, zorder=5)
ax.arrow(circle_center_x + circle_radius + .005, .55,
         .85 - h_t_width/2 - .55 - .01, 0.,
         head_width=0.025, head_length=0.025, fc='k', ec='k')

ax.axis(axis)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
ax.axis('off')

plt.savefig('6-attention_schematic.pdf', transparent=True, bbox_inches='tight', pad_inches=0.1)

In [ ]:
import lasagne
import theano
import deepdish
import glob
import sys
sys.path.append('/Users/craffel/Documents/projects/midi-dataset/')
import experiment_utils
import whoosh_search
import feature_extraction
RESULTS_PATH = '/Users/craffel/Documents/projects/midi-dataset/results/'
import pse

In [ ]:
def compute_statistics(gram):
    '''
    Computes the mean and standard deviation of feature dimensions in a
    provided feature vector sequence.
    Parameters
    ----------
    gram : np.ndarray
        Constant-Q spectrogram, shape=(n_frames, n_frequency_bins)
    Returns
    -------
    statistics : np.ndarray
        Stacked mean and standard deviation.
    '''
    # Compute and stack statistics, adding a "n_samples" dimension in front
    return np.concatenate((gram.mean(axis=0), gram.std(axis=0)))[np.newaxis]


In [ ]:
# Load in all parameter optimization trials
trial_files = glob.glob(os.path.join(
    RESULTS_PATH, 'pse_parameter_trials', '*.h5'))
trials = [deepdish.io.load(f) for f in trial_files]
# Get the hyperparameters for the trial with the lowest objective value
best_trial = sorted(trials, key=lambda t: t['best_objective'])[0]
hyperparameters = best_trial['hyperparameters']
# Load in the pre-trained parameters for the best performing models
network_params = deepdish.io.load(
    os.path.join(RESULTS_PATH, 'pse_model', 'best_model.h5'))
compute_output = {}
layers = {}
# Build networks and output-computing functions
for dataset, network in zip(['clean_midi', 'msd'], ['X', 'Y']):
    # Construct the network according to best-trial hyperparameters
    if hyperparameters['network'] == 'pse_big_filter':
        build_network = experiment_utils.build_pse_net_big_filter
    elif hyperparameters['network'] == 'pse_small_filters':
        build_network = experiment_utils.build_pse_net_small_filters
    # PSE trials do not have this hyperparameter by default, so as in
    # experiment_utils.run_trial, we must set the default value
    hyperparameters['downsample_frequency'] = hyperparameters.get(
        'downsample_frequency', True)        
    layers[network] = build_network(
        (None, 1, None, feature_extraction.N_NOTES),
        # We will supply placeholders here but load in the values below
        np.zeros((1, feature_extraction.N_NOTES), theano.config.floatX),
        np.ones((1, feature_extraction.N_NOTES), theano.config.floatX),
        hyperparameters['downsample_frequency'],
        hyperparameters['n_attention'], hyperparameters['n_conv'])
    # Load in network parameter values
    lasagne.layers.set_all_param_values(
        layers[network][-1], network_params[network])
    # Compile function for computing the output of the network
    compute_output[network] = theano.function(
        [layers[network][0].input_var],
        lasagne.layers.get_output(layers[network][-1], deterministic=True))

In [ ]:
validation_path = os.path.join(RESULTS_PATH, 'training_dataset_unaligned', 'validate', 'h5')
validation_raw = [deepdish.io.load(f) for f in glob.glob(os.path.join(validation_path, '*.h5'))]
validation_raw = [dict((k, d[k][0]) for k in ['X', 'Y']) for d in validation_raw]

In [ ]:
validation_stats = [dict((k, compute_statistics(d[k])) for k in ['X', 'Y'])
                    for d in validation_raw]

In [ ]:
validation_out = [dict((k, compute_output[k](d[k].reshape(1, 1, *d[k].shape))) for k in ['X', 'Y'])
                  for d in validation_raw]

In [ ]:
plt.figure(figsize=(FIGSIZE[0], .4*FIGSIZE[0]/.7))
with sns.axes_style('white'):
    ax = plt.gca()
    eps = .001
    ax.add_patch(matplotlib.patches.Rectangle((.3, .85), .4 + eps, .15, fc='None'))
    ax.imshow(validation_raw[100]['Y'].T,
              interpolation='nearest', aspect='auto', cmap=plt.cm.hot,
              origin='lower', extent=(.3, .7, .85, 1.),
              vmin=np.percentile(validation_raw[100]['Y'], 1),
              vmax=np.percentile(validation_raw[100]['Y'], 99))

    for n in xrange(32):
        ax.add_patch(matplotlib.patches.Rectangle((.2 + .5*(n/31.) - (.7 - .3)/8, .7 - .03*(n/31.)),
                                                  (.7 - .3)/2, (.15 - 0.)/2, fc=ORANGE))
    ax.add_patch(matplotlib.patches.Polygon([[.3, .85],
                                             [.2 - (.7 - .3)/8, .7 + (.15 - 0.)/2],
                                             [.7 - (.7 - .3)/8 + (.7 - .3)/2 - eps, .74],
                                             [.7 - (.7 - .3)/8 + (.7 - .3)/2 - eps, .7 - .03 + (.15 - 0.)/2],
                                             [.7, .85]], ec='none', fc=BLUE, zorder=-1, alpha=.8))
    ax.annotate(s=u'3×3 convolution, 2×2 max-pool',
                xy=(.49, .81), fontsize=16, ha='center', va='center', family='sans-serif')

    ax.add_patch(matplotlib.patches.Polygon([[.2 - (.7 - .3)/8, .7],
                                             [.35 - .01, .611],
                                             [.35 - .005, .58],
                                             [.35 + .005, .58],
                                             [.35 + .01, .611],
                                             [.7 - (.7 - .3)/8 + (.7 - .3)/2, .7 - .03],
                                             [.2 - (.7 - .3)/8, .72]],
                                            ec='none', fc='#CCCCCC', zorder=-1, alpha=.8))
    ax.annotate(s=u'Feedforward Attention',
                xy=(.28, .655), fontsize=16, ha='left', va='center', family='sans-serif')
    
    ax.add_patch(matplotlib.patches.Rectangle((.35 - .005, .43), .01, .15, fc=ORANGE, ec='k'))

    ax.add_patch(matplotlib.patches.Polygon([[.35 + .005, .43],
                                             [.45 - .005, .40],
                                             [.45 - .005, .61],
                                             [.35 + .005, .58]],
                                            ec='none', fc=GREEN, zorder=-1, alpha=.8))

    ax.annotate(s=u'Dense\nReLU',
                xy=(.4, .505), fontsize=16, ha='center', va='center', family='sans-serif', rotation=90)

    ax.add_patch(matplotlib.patches.Rectangle((.45 - .005, .4), .01, .21, fc=ORANGE, ec='k'))

    ax.add_patch(matplotlib.patches.Polygon([[.45 + .005, .4],
                                             [.55 - .005, .4],
                                             [.55 - .005, .61],
                                             [.45 + .005, .61]],
                                            ec='none', fc=GREEN, zorder=-1, alpha=.8))

    ax.add_patch(matplotlib.patches.Rectangle((.55 - .005, .4), .01, .21, fc=ORANGE, ec='k'))

    ax.annotate(s=u'Dense\nReLU',
                xy=(.5, .505), fontsize=16, ha='center', va='center', family='sans-serif', rotation=90)

    ax.add_patch(matplotlib.patches.Polygon([[.55 + .005, .4],
                                             [.65 - .005, .455],
                                             [.65 - .005, .555],
                                             [.55 + .005, .61]],
                                            ec='none', fc=GREEN, zorder=-1, alpha=.8))
    ax.annotate(s=u'Dense\ntanh',
                xy=(.6, .505), fontsize=16, ha='center', va='center', family='sans-serif', rotation=90)

    ax.imshow(np.array([np.random.random_sample(24)*.5]).T, cmap=plt.cm.hot,
              extent=(.65 - .005, .65 + .005, .455, .555), interpolation='nearest')
    ax.add_patch(matplotlib.patches.Rectangle((.65 - .005, .455), .01 + eps, .1, fc='None'))

    plt.axis('off')
    plt.plot([0, 1], [0, 1], alpha=0.)
    plt.xlim([0.15 - eps, 0.85 + .005])
    plt.ylim([.4 - eps, 1 + eps])
    if False:
        for n in np.linspace(0, 1, 21):
            plt.plot([0, 1], [n, n], 'k:')
            plt.plot([n, n], [0, 1], 'k:')
            ax.annotate(s=str(n),
                        xy=(0., n), ha='center', va='center', family='sans-serif')
            ax.annotate(s=str(n),
                        xy=(n, 1.), ha='center', va='center', family='sans-serif')
    plt.savefig('6-model_schematic.pdf', transparent=True, bbox_inches='tight', pad_inches=0.1)


In [ ]:
validation_raw_0 = validation_raw[229]
validation_raw_1 = validation_raw[339]

X_embed_mean = np.mean(np.concatenate([o['X'] for o in validation_out]), axis=0)
Y_embed_mean = np.mean(np.concatenate([o['Y'] for o in validation_out]), axis=0)

with sns.axes_style('white'):
    fig = plt.figure(figsize=(FIGSIZE[0], 3))
    gs1 = matplotlib.gridspec.GridSpec(
        2, 2, width_ratios=[validation_raw_0['X'].shape[0],
                            validation_raw_0['Y'].shape[0]])
    gs1.update(left=.04, right=.97, top=1., bottom=.37, wspace=.02, hspace=.1)

    cqt_spacing = 300

    ax = plt.subplot(gs1[0])
    ax.imshow(validation_raw_0['X'].T, aspect='auto', origin='lower',
              interpolation='nearest', cmap=plt.cm.hot,
              vmin=np.percentile(validation_raw_0['X'], 1),
              vmax=np.percentile(validation_raw_0['X'], 99))
    ax.set_xticks([])
    ax.set_yticks(np.arange(0, validation_raw_0['X'].shape[1], 12))
    ax.set_yticklabels(['C{}'.format(n) for n in [3, 4, 5, 6]])
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')

    ax = plt.subplot(gs1[1])
    ax.imshow(validation_raw_0['Y'].T, aspect='auto', origin='lower',
              interpolation='nearest', cmap=plt.cm.hot,
              vmin=np.percentile(validation_raw_0['Y'], 1),
              vmax=np.percentile(validation_raw_0['Y'], 99))
    ax.set_xticks([])
    ax.set_yticks([])
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')

    gs3 = matplotlib.gridspec.GridSpec(
        1, 2, width_ratios=[validation_raw_0['X'].shape[0],
                            validation_raw_0['Y'].shape[0]])
    gs3.update(top=.46, bottom=.11, wspace=.02, hspace=.5)

    cqt_spacing = 600

    ax = plt.subplot(gs1[2])
    ax.imshow(validation_raw_1['X'].T, aspect='auto', origin='lower',
              interpolation='nearest', cmap=plt.cm.hot,
              vmin=np.percentile(validation_raw_1['X'], 1),
              vmax=np.percentile(validation_raw_1['X'], 99))
    ax.set_xticks(np.arange(0, validation_raw_1['X'].shape[0], cqt_spacing))
    ax.set_yticks(np.arange(0, validation_raw_0['X'].shape[1], 12))
    ax.set_yticklabels(['C{}'.format(n) for n in [3, 4, 5, 6]])
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')

    ax = plt.subplot(gs1[3])
    ax.imshow(validation_raw_1['Y'].T, aspect='auto', origin='lower',
              interpolation='nearest', cmap=plt.cm.hot,
              vmin=np.percentile(validation_raw_1['Y'], 1),
              vmax=np.percentile(validation_raw_1['Y'], 99))
    ax.set_xticks(np.arange(0, validation_raw_0['Y'].shape[0], cqt_spacing))
    ax.set_yticks([])
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')

    fig.text(0.5, 0.27, 'Frame Index', ha='center', va='center')
    fig.text(0.0, 0.68, 'Frequency', ha='center', va='center', rotation='vertical')

    embed_spacing = 16
    gs2 = matplotlib.gridspec.GridSpec(2, 1)
    gs2.update(left=.04, right=.97, top=.22, bottom=.125, wspace=.03, hspace=.2)

    X_val_embed0 = validation_out[229]['X'] - X_embed_mean
    Y_val_embed0 = validation_out[229]['Y'] - Y_embed_mean
    X_val_embed1 = validation_out[339]['X'] - X_embed_mean
    Y_val_embed1 = validation_out[339]['Y'] - Y_embed_mean
    print 'Pairwise distances:'
    print scipy.spatial.distance.cdist(
        np.concatenate([X_val_embed0, X_val_embed1]),
        np.concatenate([Y_val_embed0, Y_val_embed1]), 'sqeuclidean')

    vmin = min([np.percentile(x, 2) for x in [X_val_embed0, X_val_embed1, Y_val_embed0, Y_val_embed1]])
    vmax = max([np.percentile(x, 98) for x in [X_val_embed0, X_val_embed1, Y_val_embed0, Y_val_embed1]])

    ax = plt.subplot(gs2[0])
    ax.imshow(X_val_embed0, aspect='auto', origin='lower',
              interpolation='nearest', cmap=plt.cm.hot,
              vmin=vmin, vmax=vmax)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.xaxis.set_ticks_position('bottom')

    ax = plt.subplot(gs2[1])
    ax.imshow(Y_val_embed0, aspect='auto', origin='lower',
              interpolation='nearest', cmap=plt.cm.hot,
              vmin=vmin, vmax=vmax)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.xaxis.set_ticks_position('bottom')

    gs3 = matplotlib.gridspec.GridSpec(2, 1)
    gs3.update(left=.04, right=.97, top=.095, bottom=0., wspace=.03, hspace=.2)

    ax = plt.subplot(gs3[0])
    ax.imshow(X_val_embed1, aspect='auto', origin='lower',
              interpolation='nearest', cmap=plt.cm.hot,
              vmin=vmin, vmax=vmax)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.xaxis.set_ticks_position('bottom')

    ax = plt.subplot(gs3[1])
    ax.imshow(Y_val_embed1, aspect='auto', origin='lower',
              interpolation='nearest', cmap=plt.cm.hot,
              vmin=vmin, vmax=vmax)
    ax.set_xticks(range(0, X_val_embed0.shape[1], embed_spacing) + [127])
    ax.set_xticklabels([1] + range(embed_spacing, X_val_embed0.shape[1], embed_spacing) + [128])
    ax.set_yticks([])
    ax.xaxis.set_ticks_position('bottom')

    ax = fig.add_axes([0., 0., 1., 1.], frameon=False)
    ax.axis('off')
    ax.add_patch(matplotlib.patches.FancyArrowPatch(
        (0.04, 0.83),
        (0.04, 0.18),
        connectionstyle='arc3, rad=0.25',
        alpha=.6, fc='k', ls='dashed',
        mutation_scale=15,
        arrowstyle='-|>'))

    ax.add_patch(matplotlib.patches.FancyArrowPatch(
        (0.97, 0.83),
        (0.97, 0.13),
        connectionstyle='arc3, rad=-0.25',
        alpha=.6, fc='k', ls='dashed',
        mutation_scale=15,
        arrowstyle='-|>'))

    ax.add_patch(matplotlib.patches.FancyArrowPatch(
        (0.04, 0.49),
        (0.04, 0.05),
        connectionstyle='arc3, rad=0.3',
        alpha=.6, fc='k', ls='dashed',
        mutation_scale=15,
        arrowstyle='-|>'))

    ax.add_patch(matplotlib.patches.FancyArrowPatch(
        (0.97, 0.49),
        (0.97, 0.0),
        connectionstyle='arc3, rad=-0.3',
        alpha=.6, fc='k', ls='dashed',
        mutation_scale=15,
        arrowstyle='-|>'))

    plt.savefig('6-embeddings.pdf', transparent=True, bbox_inches='tight', pad_inches=0.)

In [ ]:
fig = plt.figure(figsize=(FIGSIZE[0], 2*FIGSIZE[1]), )
min_dist = .0025
max_dist = .5
for n, rep in enumerate([validation_out, validation_stats]):
    all_X = np.concatenate([d['X'] for d in rep])
    all_Y = np.concatenate([d['Y'] for d in rep])
    shuffle = pse.random_derangement(all_Y.shape[0])
    all_Y_n = all_Y[shuffle]
    p_distances = np.sum((all_X - all_Y)**2, axis=1)
    n_distances = np.sum((all_X - all_Y_n)**2, axis=1)
    colors = [BLUE, GREEN]
    #ax = plt.subplot(2, 1, n + 1, )
    gs = matplotlib.gridspec.GridSpec(2, 1, hspace=0.08)
    ax = plt.subplot(gs[n, 0])
    ax.hist(p_distances, bins=10**np.linspace(np.log10(min_dist), np.log10(max_dist)),
             fc=colors[0], alpha=.7, weights=np.ones(p_distances.shape[0])/p_distances.shape[0])
    ax.hist(n_distances, bins=10**np.linspace(np.log10(min_dist), np.log10(max_dist)),
             fc=colors[1], alpha=.7, weights=np.ones(n_distances.shape[0])/n_distances.shape[0])
    ax.set_xscale("log")
    ax.set_xlim(.0025, .5)
    plt.xticks([.005, .01, .025, .05, .1, .25], [.005, .01, .025, .05, .1, .25])
    if n == 0:
        ax.set_ylim(0, 0.1)
    else:
        plt.xlabel('Distance')    
        ax.set_ylim(0, 0.105)
    plt.ylabel('Proportion')
        
#fig.text(0.5, 0.085, 'Distance', ha='center', va='center')
#fig.text(0.065, 0.5, 'Proportion', ha='center', va='center', rotation='vertical')
        
plt.savefig('6-distributions.pdf',  bbox_inches='tight', pad_inches=0.1)

In [ ]:
rank_curves = {}
for rep in ['pse', 'stats']:
    results = deepdish.io.load(os.path.join(RESULTS_PATH, '{}_match_results/test_results.h5'.format(rep)))
    rank_curves[rep] = prop_below(results)
    ranks = [min(r['msd_match_ranks']) for r in results]
    print '{} MRR: {}, rank 1: {}, rank for < 95%: {}'.format(
        rep,
        np.mean([1./(r + 1) for r in ranks]),
        np.mean(np.less(ranks, 1)),
        np.argmax(rank_curves[rep] > .95) + 1)

In [ ]:
plt.figure(figsize=FIGSIZE)
for rep, color in zip(['pse', 'stats'], [BLUE, GREEN]):
    plt.semilogx(100*rank_curves[rep], color, lw=2)
plt.ylim(0, 102)
plt.ylabel('Percentage below')
plt.xlabel('Rank')
plt.text(4000, 75, 'PSE', {'color': BLUE, 'weight': 'bold', 'size': 16})
plt.text(24000, 65, 'Statistics', {'color': GREEN, 'weight': 'bold', 'size': 16})
plt.text(24000, 65, 'Statistics', {'color': 'k', 'weight': 'bold', 'size': 16}, alpha=.1)
plt.savefig('6-ranks.pdf',  bbox_inches='tight', pad_inches=0.1)    